This notebook is part of [**Byron v0.1**](https://github.com/squillero/byron)  
Copyright 2023 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: [Apache-2.0](https://www.tldrlegal.com/license/apache-license-2-0-apache-2-0) 

In [1]:
import math
from pprint import pprint

# Macros & Parameters

In [2]:
import byron

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_49336/826285489.py:1: RuntimeWarning: 
  Paranoia checks are enabled in this notebook: performances can be significantly impaired
  [see https://github.com/squillero/byron/blob/pre-alpha/docs/paranoia.md for details]
  import byron


Everything under the namespace `byron.framework` (lazy fingers can use `byron.f`).

## Parameters

In [3]:
byte = byron.f.integer_parameter(0, 2**8)
almost_pi = byron.f.float_parameter(21 * math.sin(math.pi / 21), 42 * math.tan(math.pi / 42))
register = byron.f.choice_parameter(['ax', 'bx', 'cx', 'dx'])
bitmask = byron.f.array_parameter("01-", 32)

Parameters are *types* (classes), that is, all the above functions are technically *factories*. 

In [4]:
byte

byron.parameter.Int[8bit]

Parameters are used by Byron to create objects (*instances*). The initial (non initialized) value of these objects is always `None`; parameter instances are initialized with the first mutation and then, possibly, mutated again.

**Notez bien:** Users do not need to use *parameters* directly. 

In [5]:
for param in [byte, almost_pi, register, bitmask]:
    obj = param()
    initial_value = obj.value
    obj.mutate()
    value = obj.value
    print(f"{obj!r} :: {initial_value!r} -> {value!r}")

<byron.parameter.Int[8bit] object at 0x162037510> :: None -> 198
<byron.parameter.Float[3.1298875896996634–3.1474648808016528) object at 0x162036a10> :: None -> 3.137601883793572
<byron.parameter.Choice[ax┊bx┊cx┊dx] object at 0x1151236d0> :: None -> 'bx'
<byron.parameter.Array[-01ｘ32] object at 0x162037510> :: None -> '1-1--0111110-1000-11001000--01-1'


Instances of `byron.parameter` can be printed using f-string [syntax](https://docs.python.org/3/reference/lexical_analysis.html#f-strings)

In [6]:
num = byte()
num.mutate()
print(f"Default: {num}\nRepr: {num!r}\nBinary (16bit): {num:016b}\nOctal: {num:o}\n#Hex: {num:#x}")

Default: 161
Repr: <byron.parameter.Int[8bit] object at 0x162035310>
Binary (16bit): 0000000010100001
Octal: 241
#Hex: 0xa1


## Aliases (shared parameters) 

Parameters are classes (types), thus objects created are independent. 

In [7]:
parameter = byron.f.integer_parameter(0, 10_000)

var1 = parameter()
var2 = parameter()
var3 = parameter()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  Initial")
var1.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var1")
var2.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var2")
var3.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var3")

var1: None // var2: None // var3: None  --  Initial
var1: 7580 // var2: None // var3: None  --  After mutating var1
var1: 7580 // var2: 3545 // var3: None  --  After mutating var2
var1: 7580 // var2: 3545 // var3: 9706  --  After mutating var3


However, it may be necessary to have all instances of a specific parameter sharing the very same data. Parameters defined by a `make_shared_parameter` behave like *aliases*: they always have the same value, changes (e.g., `mutate()`) only affect the *first* instance.

In [8]:
shared_parameter1 = byron.f.make_shared_parameter(byron.f.integer_parameter(0, 10_000))
shared_parameter2 = byron.f.make_shared_parameter(byron.f.integer_parameter(0, 10_000))

var1 = shared_parameter1()
var2 = shared_parameter1()
var3 = shared_parameter2()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  Initial")
var1.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var1")
var2.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var2")
var3.mutate()
print(f"var1: {var1} // var2: {var2} // var3: {var3}  --  After mutating var3")

var1: None // var2: None // var3: None  --  Initial
var1: 8931 // var2: 8931 // var3: None  --  After mutating var1
var1: 8931 // var2: 8931 // var3: None  --  After mutating var2
var1: 8931 // var2: 8931 // var3: 7783  --  After mutating var3


## Notes

### Meaningful names

In [9]:
byron.f.integer_parameter(0, 100), byron.f.integer_parameter(0, 256)

(byron.parameter.Int[0..99], byron.parameter.Int[8bit])

In [10]:
byron.f.choice_parameter("BAGFECD")

byron.parameter.Choice[A┊B┊C┊D┊E┊F┊G]

### Helpful warnings

Friendly suggestions are shown if code is not optimized and logging level is `DEBUG` (the default in notebooks)

In [11]:
possible_error = byron.f.integer_parameter(1, 100)

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_49336/2539362671.py:1: SyntaxWarning: Friendly suggestion:
  Parameter ranges are half-open: the maximum value is 99 (ie. a range of 99 possible values) — did you mean '(1, 101)'?
  possible_error = byron.f.integer_parameter(1, 100)


In [12]:
possible_error = byron.f.choice_parameter(range(1_000))

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_49336/2234211310.py:1: SyntaxWarning: Friendly suggestion:
  Choice parameters with many alternatives impair performances — why not using an integer parameter [0-1000)?
  possible_error = byron.f.choice_parameter(range(1_000))


### Cache

Results are cached. Calling the same parameter factory with the same arguments yields the same parameter (the **very same** one, not an identical copy)

In [13]:
word1 = byron.f.integer_parameter(0, 2**16)
word2 = byron.f.integer_parameter(0, 65_536)

word1 == word2, word1 is word2

(True, True)

In [14]:
choice1 = byron.f.choice_parameter(['Yes', 'No', 'Maybe'])
choice2 = byron.f.choice_parameter(('Maybe', 'Yes', 'No'))

choice1 == choice2, choice1 is choice2

(True, True)

# Macros

Macros represent multi-line parametric fragments. Macros, as a Frames, can be tested using `show`.

In [15]:
macro = byron.f.macro(
    "{var} := {num}", var=byron.f.choice_parameter(['foo', 'bar']), num=byron.f.integer_parameter(-100, 100)
)
byron.f.show(macro)  # use node_info=False to remove node info "🖋 n0.n1 ➜ "

foo := 54  ; 🖋 n1 ➜ Macro❬User#1❭



In [16]:
variable = byron.f.choice_parameter(['foo', 'bar'])
integer = byron.f.integer_parameter(-100, 100)
macro = byron.f.macro("{var} := {num:#x} ; ie. set the variable '{var}' to {num}", var=variable, num=integer)
byron.f.show(macro)

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭



# `byron.f.show`

In [21]:
for _ in range(5):
    byron.f.show(macro)

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭

foo := 0x36 ; ie. set the variable 'foo' to 54  ; 🖋 n1 ➜ Macro❬User#2❭



In [24]:
for _ in range(5):
    byron.f.show(macro, node_info=False, seed=None)

bar := -0x5f ; ie. set the variable 'bar' to -95

bar := -0x50 ; ie. set the variable 'bar' to -80

foo := -0x56 ; ie. set the variable 'foo' to -86

bar := 0x2 ; ie. set the variable 'bar' to 2

foo := -0x1f ; ie. set the variable 'foo' to -31

